In [42]:
#Import data, cleaning data, create sales insight and market basket analysis 

In [43]:
import pandas as pd
import os
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import operator as op
import plotly.express as px
from mlxtend.frequent_patterns import apriori, association_rules
import locale

#upload CSV file
file_path = 'C:\capstoneproject\groceries_shopping.csv'

# Read the CSV file
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

   memberID        Date          itemName  quantity  price
0    1808.0  21-07-2015    tropical fruit        30   4298
1    2552.0  05-01-2015        whole milk         3   6214
2    2300.0  19-09-2015         pip fruit        29   9893
3    1187.0  12-12-2015  other vegetables        23   7663
4    3037.0  01-02-2015        whole milk        13   2052


<>:14: SyntaxWarning:

invalid escape sequence '\c'

<>:14: SyntaxWarning:

invalid escape sequence '\c'

C:\Users\jorda\AppData\Local\Temp\ipykernel_9020\2731620911.py:14: SyntaxWarning:

invalid escape sequence '\c'



In [44]:
# To check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   memberID  38717 non-null  float64
 1   Date      38765 non-null  object 
 2   itemName  38749 non-null  object 
 3   quantity  38765 non-null  int64  
 4   price     38765 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.5+ MB


In [45]:
# Renaming the columns, because it would be easier to understand what that means.
df.columns = ['memberID', 'Date', 'itemName', 'quantity', 'Price']
df.head()

,memberID,Date,itemName,quantity,Price
0,1808.0,21-07-2015,tropical fruit,30,4298
1,2552.0,05-01-2015,whole milk,3,6214
2,2300.0,19-09-2015,pip fruit,29,9893
3,1187.0,12-12-2015,other vegetables,23,7663
4,3037.0,01-02-2015,whole milk,13,2052


In [46]:
# Checking for the missing values
nan_values = df.isna().sum()
nan_values

memberID    48
Date         0
itemName    16
quantity     0
Price        0
dtype: int64

In [47]:
# there are missing values
df = df.dropna(subset=['memberID','itemName'])

# Converting Date column into correct datatype which is datetime
df.Date = pd.to_datetime(df.Date)
df.memberID = df['memberID'].astype('str')
df.info()  # They are in correct datatype now

<class 'pandas.core.frame.DataFrame'>
Index: 38701 entries, 0 to 38764
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   memberID  38701 non-null  object        
 1   Date      38701 non-null  datetime64[ns]
 2   itemName  38701 non-null  object        
 3   quantity  38701 non-null  int64         
 4   Price     38701 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 1.8+ MB


C:\Users\jorda\AppData\Local\Temp\ipykernel_9020\2154220146.py:5: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [48]:
#create new column to calculate total price
df['totalPrice'] = df['quantity'] * df['price']

# Display the first few rows to verify the new column
df.head()

,memberID,Date,itemName,quantity,Price,totalPrice
0,1808.0,2015-07-21,tropical fruit,30,4298,128940
1,2552.0,2015-01-05,whole milk,3,6214,18642
2,2300.0,2015-09-19,pip fruit,29,9893,286897
3,1187.0,2015-12-12,other vegetables,23,7663,176249
4,3037.0,2015-02-01,whole milk,13,2052,26676


In [49]:
Sales_weekly = df.resample('w', on='Date').size()
fig = px.line(df, x=Sales_weekly.index, y=Sales_weekly,
              labels={'y': 'Number of Sales',
                     'x': 'Date'})
fig.update_layout(title_text='Number of Sales Weekly',
                  title_x=0.5, title_font=dict(size=18)) 
fig.show()

C:\Users\jorda\AppData\Local\Temp\ipykernel_9020\94575179.py:1: FutureWarning:

'w' is deprecated and will be removed in a future version, please use 'W' instead.



In [50]:
Unique_customer_weekly = df.resample('w', on='Date').memberID.nunique()
fig = px.line(Unique_customer_weekly, x=Unique_customer_weekly.index, y=Unique_customer_weekly,
              labels={'y': 'Number of Customers'})
fig.update_layout(title_text='Number of Customers Weekly',
                  title_x=0.5, title_font=dict(size=18))
fig.show()

C:\Users\jorda\AppData\Local\Temp\ipykernel_9020\2337930868.py:1: FutureWarning:

'w' is deprecated and will be removed in a future version, please use 'W' instead.



In [51]:
Sales_per_Customer = Sales_weekly / Unique_customer_weekly
fig = px.line(Sales_per_Customer, x=Sales_per_Customer.index, y=Sales_per_Customer,
              labels={'y': 'Sales per Customer Ratio'})
fig.update_layout(title_text='Sales per Customer Weekly',
                  title_x=0.5, title_font=dict(size=18))
fig.update_yaxes(rangemode="tozero")
fig.show()

In [52]:
#Frequency_of_items = df.groupby(pd.Grouper(key='itemName')).size().reset_index(name='count')
Frequency_of_items = df.groupby('itemName')['quantity'].sum().reset_index(name='quantitySold')

#Top 20% item sold calculation. Use the Pareto rules
threshold = Frequency_of_items['quantitySold'].quantile(0.80)

# Filter the DataFrame to include only items above this threshold
top_items = Frequency_of_items[Frequency_of_items['quantitySold'] >= threshold]

fig = px.treemap(top_items, path=['itemName'], values='quantitySold')
fig.update_layout(title_text='Frequency of the Items Sold',
                  title_x=0.5, title_font=dict(size=18)
                  )
fig.update_traces(textinfo="label+value")
fig.show()

In [53]:
# Sort the DataFrame by quantitySold in descending order
top_items = top_items.sort_values(by='quantitySold', ascending=False)

# Select the top 10 items
top20_items = top_items.head(10)

# Create the treemap with the top 10 items
fig = px.treemap(top20_items, path=['itemName'], values='quantitySold')
fig.update_layout(title_text='Top 10 Items Sold',
                  title_x=0.5, title_font=dict(size=18))
fig.update_traces(textinfo="label+value")
fig.show()

In [76]:
user_item = df.groupby(pd.Grouper(key='memberID')).size().reset_index(name='count')
fig = px.bar(user_item.head(25), x='memberID', y='count',
             labels={'y': 'Number of Sales',
                     'count': 'Number of Items Bought'},
             color='count')
fig.update_layout(title_text='Top 20 Customers regarding Number of Items Bought',
                  title_x=0.5, title_font=dict(size=18))
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.show()

In [81]:
# Define the order of the days
days_order = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat', 'Sabtu', 'Minggu']

# Group by day of the week and count the items
day = df.groupby(df['Date'].dt.strftime('%A'))['itemName'].count()

# Convert the index to a categorical type with the specified order
day.index = pd.Categorical(day.index, categories=days_order, ordered=True)

# Sort the values by the categorical index
day = day.sort_index()

fig = px.bar(day, x=day.index, y=day, color=day,
             labels={'y': 'Number of Sales',
                     'Date': 'Week Days'})
fig.update_layout(title_text='Number of Sales per Discrete Week Days',
                  title_x=0.5, title_font=dict(size=18))
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.show()

In [56]:
#create market basket analysis
baskets = df.groupby(['memberID', 'itemName'])['itemName'].count().unstack().fillna(0).reset_index()
baskets.head()

itemName,memberID,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
1,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0
2,1002.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1003.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1004.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0


In [57]:
# Let's check the most sold -item which is whole milk- has the same number of sales as we discussed above in the treemap.
baskets['whole milk'].sum()

np.float64(2500.0)

In [58]:
# Encoding the items that sold more than 1
def one_hot_encoder(k):
    if k <= 0:
        return 0
    if k >= 1:
        return 1
baskets_final = baskets.iloc[:, 1:baskets.shape[1]].applymap(one_hot_encoder)
baskets_final.head()

C:\Users\jorda\AppData\Local\Temp\ipykernel_9020\1272554220.py:7: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



itemName,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [59]:
# Finding the most frequent items sold together
frequent_itemsets = apriori(baskets_final, min_support=0.025, use_colnames=True, max_len=3).sort_values(by='support')
frequent_itemsets.head(25)

C:\Users\jorda\AppData\Roaming\Python\Python312\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



,support,itemsets
522,0.025141,"(whole milk, pastry, root vegetables)"
538,0.025141,"(shopping bags, rolls/buns, yogurt)"
476,0.025141,"(other vegetables, rolls/buns, frankfurter)"
466,0.025141,"(whole milk, citrus fruit, tropical fruit)"
411,0.025141,"(whole milk, specialty bar)"
69,0.025141,(spread cheese)
404,0.025141,"(whole milk, sliced cheese)"
85,0.025141,"(beef, pip fruit)"
118,0.025141,"(chocolate, bottled water)"
247,0.025141,"(shopping bags, domestic eggs)"


In [60]:
# Creating association rules for indicating astecedent and consequent items
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values('lift', ascending=False)
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rules.head(25)

,antecedents,consequents,support,confidence,lift
861,(sausage),"(rolls/buns, yogurt)",0.035403,0.172285,1.547386
856,"(rolls/buns, yogurt)",(sausage),0.035403,0.317972,1.547386
857,"(rolls/buns, sausage)",(yogurt),0.035403,0.432602,1.530202
860,(yogurt),"(rolls/buns, sausage)",0.035403,0.125227,1.530202
449,"(whole milk, root vegetables)",(shopping bags),0.028989,0.256818,1.528362
452,(shopping bags),"(whole milk, root vegetables)",0.028989,0.172519,1.528362
929,"(other vegetables, yogurt)",(sausage),0.036942,0.307692,1.497359
932,(sausage),"(other vegetables, yogurt)",0.036942,0.179775,1.497359
602,"(other vegetables, soda)",(shopping bags),0.030785,0.248447,1.478545
603,(shopping bags),"(other vegetables, soda)",0.030785,0.183206,1.478545


In [61]:
# This step, we will do the customer segmentation using RFM (recency, frequency, monetary) analysis

In [62]:
# Finding last purchase date of each customer - Customer Recency
Recency = df.groupby(by='memberID')['Date'].max().reset_index()
Recency.columns = ['memberID', 'LastDate']
Recency.head()

,memberID,LastDate
0,1000.0,2015-11-25
1,1001.0,2015-05-02
2,1002.0,2015-08-30
3,1003.0,2015-02-10
4,1004.0,2015-12-02


In [63]:
# Finding last date for our dataset
last_date_dataset = Recency['LastDate'].max()
last_date_dataset

Timestamp('2015-12-30 00:00:00')

In [64]:
# Calculating Recency by subtracting (last transaction date of dataset) and (last purchase date of each customer)
Recency['Recency'] = Recency['LastDate'].apply(lambda x: (last_date_dataset - x).days)
Recency.head()

,memberID,LastDate,Recency
0,1000.0,2015-11-25,35
1,1001.0,2015-05-02,242
2,1002.0,2015-08-30,122
3,1003.0,2015-02-10,323
4,1004.0,2015-12-02,28


In [65]:
fig = px.histogram(Recency, x='Recency', opacity=0.85, marginal='box')
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Recency Distribution of the Customers',
                  title_x=0.5, title_font=dict(size=20))
fig.show()

In [66]:
# Frequency of the customer visits
Frequency = df.drop_duplicates(['Date', 'memberID']).groupby(by=['memberID'])['Date'].count().reset_index()
Frequency.columns = ['memberID', 'Visit_Frequency']
Frequency.head()

,memberID,Visit_Frequency
0,1000.0,5
1,1001.0,5
2,1002.0,4
3,1003.0,4
4,1004.0,8


In [67]:
fig = px.histogram(Frequency, x='Visit_Frequency', opacity=0.85, marginal='box')
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Visit Frequency Distribution of the Customers',
                  title_x=0.5, title_font=dict(size=20))
fig.show()

In [68]:
# Group by memberID and sum totalPrice
Monetary = df.groupby(by="memberID")['totalPrice'].sum().reset_index()
Monetary.columns = ['memberID', 'Monetary']

# Set locale to Indonesian
locale.setlocale(locale.LC_ALL, 'id_ID.UTF-8')

# Format the monetary values as Rupiah currency
Monetary['Monetary'] = Monetary['Monetary'].apply(lambda x: locale.currency(x, grouping=True))

Monetary.head() 

,memberID,Monetary
0,1000.0,"Rp1.278.166,00"
1,1001.0,"Rp1.037.705,00"
2,1002.0,"Rp519.701,00"
3,1003.0,"Rp598.451,00"
4,1004.0,"Rp1.838.625,00"


In [69]:
fig = px.histogram(Monetary, x='Monetary', opacity=0.85, marginal='box',
                   labels={'itemName': 'Monetary'})
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Monetary Distribution of the Customers',
                  title_x=0.5, title_font=dict(size=20))
fig.show()

In [70]:
# Combining all scores into one DataFrame
RFM = pd.concat([Recency['memberID'], Recency['Recency'], Frequency['Visit_Frequency'], Monetary['Monetary']], axis=1)
RFM.head()

,memberID,Recency,Visit_Frequency,Monetary
0,1000.0,35,5,"Rp1.278.166,00"
1,1001.0,242,5,"Rp1.037.705,00"
2,1002.0,122,4,"Rp519.701,00"
3,1003.0,323,4,"Rp598.451,00"
4,1004.0,28,8,"Rp1.838.625,00"


In [71]:
# RFM scoring
# 5-5 score = the best customers
RFM['Recency_quartile'] = pd.qcut(RFM['Recency'], 5, [5, 4, 3, 2, 1])
RFM['Frequency_quartile'] = pd.qcut(RFM['Visit_Frequency'], 5, [1, 2, 3, 4, 5])

RFM['RF_Score'] = RFM['Recency_quartile'].astype(str) + RFM['Frequency_quartile'].astype(str)
RFM.head()

,memberID,Recency,Visit_Frequency,Monetary,Recency_quartile,Frequency_quartile,RF_Score
0,1000.0,35,5,"Rp1.278.166,00",5,4,54
1,1001.0,242,5,"Rp1.037.705,00",2,4,24
2,1002.0,122,4,"Rp519.701,00",3,3,33
3,1003.0,323,4,"Rp598.451,00",2,3,23
4,1004.0,28,8,"Rp1.838.625,00",5,5,55


In [72]:
segt_map = {  # Segmentation Map [Ref]
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at risk',
    r'[1-2]5': 'can\'t loose',
    r'3[1-2]': 'about to sleep',
    r'33': 'need attention',
    r'[3-4][4-5]': 'loyal customers',
    r'41': 'promising',
    r'51': 'new customers',
    r'[4-5][2-3]': 'potential loyalists',
    r'5[4-5]': 'champions'
}

RFM['RF_Segment'] = RFM['RF_Score'].replace(segt_map, regex=True)
RFM.head()

,memberID,Recency,Visit_Frequency,Monetary,Recency_quartile,Frequency_quartile,RF_Score,RF_Segment
0,1000.0,35,5,"Rp1.278.166,00",5,4,54,champions
1,1001.0,242,5,"Rp1.037.705,00",2,4,24,at risk
2,1002.0,122,4,"Rp519.701,00",3,3,33,need attention
3,1003.0,323,4,"Rp598.451,00",2,3,23,at risk
4,1004.0,28,8,"Rp1.838.625,00",5,5,55,champions


In [73]:
x = RFM.RF_Segment.value_counts()
fig = px.treemap(x, path=[x.index], values=x)
fig.update_layout(title_text='Distribution of the RFM Segments', title_x=0.5,
                  title_font=dict(size=20))
fig.update_traces(textinfo="label+value+percent root")
fig.show()

In [74]:
fig = px.scatter(RFM, x="Visit_Frequency", y="Recency", color='RF_Segment',
                 labels={"math score": "Math Score",
                         "writing score": "Writing Score"})
fig.update_layout(title_text='Relationship between Visit_Frequency and Recency',
                  title_x=0.5, title_font=dict(size=20))
fig.show()